In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
#Imports
from __future__ import absolute_import
from __future__ import print_function
from __future__ import print_function
from __future__ import absolute_import
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.utils import np_utils, generic_utils
from keras import backend as K
#from tensorflow.keras import backend as k
#from tensorflow.python.keras import backend as k
from keras.layers.core import Lambda
from six.moves import range
import numpy as np
import scipy as sp
#from keras import backend as K
import random
import scipy.io
import matplotlib.pyplot as plt
from keras.regularizers import l2
import math
import numpy as np
import sys
import random
import warnings
import pprint
from six.moves import range
import six
import time
import os
import threading
from tensorflow.python.framework import ops
try:
    import queue
except ImportError:
    import Queue as queue
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.core import Dense, Dropout, Activation, Flatten

#Importing 5' Data
# Imports and functions from 5' Model
#%matplotlib inline
from matplotlib.colors import LogNorm
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D,Input, Flatten
from keras import models
from keras import optimizers
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import numpy as np
import time
import sys
import six

nuc_arr = ['A','C','G','T']
#Function for calculating modified probability of splicing at SD1
def prob_SD1 (sd1_freq, sd2_freq):
    if (sd1_freq==0 and sd2_freq==0):
        return 0.0
    else:
        return sd1_freq/(sd1_freq+sd2_freq)
#Function converting nucleotide sequence to numerical array with 4 channels
def seq_to_arr (seq):
    seq_len = len(seq)
    arr_rep = np.zeros((seq_len, len(nuc_arr)))
    for i in range(seq_len):
        arr_rep[i][nuc_arr.index(seq[i])] = 1
    return arr_rep

#Creating a modified dataset with only the necessary information
#Storing model inputs and outputs
reads_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911086_A5SS_spliced_reads.txt"
seq_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911085_A5SS_seq.txt"
s1_indx = 1
s2_indx= 45
seq_len = 101
read_lines = []
seq_lines = []
data_table = []
with open(reads_path) as f:
    f.readline()
    for line in f:
        mod_line = line.split('\t')
        read_lines.append([mod_line[0], mod_line[s1_indx], mod_line[s2_indx]])
with open(seq_path) as f:
    f.readline()
    for line in f:
        mod_line = line.split('\t')
        seq_lines.append([mod_line[0], mod_line[1][:-1]])

n = len(read_lines)
prob_s1 = np.zeros(n)
inputs = np.zeros((n,seq_len, 4))

for i in range(n):
    prob_s1[i] = prob_SD1(float(read_lines[i][1]), float(read_lines[i][2]))
    inputs[i] = seq_to_arr(seq_lines[i][1])



Using TensorFlow backend.


In [0]:
def standardize_X(X):
    if type(X) == list:
        return X
    else:
        return [X]
      
def permanent_dropout_layer(rate, seed):
    return Lambda(lambda x: K.dropout(x,level=rate,seed=seed))

def predict_stochastic(model, X, batch_size=128, verbose=0):
    '''Generate output predictions for the input samples
    batch by batch.
    # Arguments
        X: the input data, as a numpy array.
        batch_size: integer.
        verbose: verbosity mode, 0 or 1.f
    # Returns
        A numpy array of predictions.
    '''
    X = standardize_X(X)
    pred_Y = model.predict(X,batch_size=batch_size, verbose=verbose)
    return np.array(pred_Y)

In [5]:
np.random.seed(1)

Queries = 1

all_rmse_rand = 0
all_rmse_bald = 0
batch_size = 256
dropout_iterations = 100
X = np.array(inputs)
y = np.array(prob_s1)
all_train_indices = np.load('/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/trainindices1.npy')
size_train = 1000
train_data_indices = all_train_indices[0:size_train].tolist()
test_indices = np.load('/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/testindices1.npy')
X_train = X[train_data_indices]
y_train = y[train_data_indices]
X_test = X[test_indices]
y_test = y[test_indices]

#X_pool = X[ index_pool, :  ]
#y_pool = y[ index_pool ]
#normalise the dataset - X_train, y_train and X_test
std_X_train = np.std(X_train, 0)
std_X_train[ std_X_train == 0  ] = 1
mean_X_train = np.mean(X_train, 0)
mean_y_train = np.mean(y_train)
std_y_train = np.std(y_train)


#X_test = (X_test - mean_X_train) / std_X_train

tau = 0.159708
N = X_train.shape[0]
dropout = 0.05
seed = 38
lengthscale = 1e2
Weight_Decay = (1 - dropout) / (2. * N * lengthscale**2 * tau)
model = Sequential()
model.add(Conv1D(seq_len,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(permanent_dropout_layer(dropout,seed))
model.add(Conv1D(seq_len//2, (4), strides=1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(permanent_dropout_layer(dropout,seed))
model.add(Dense(50, W_regularizer=l2(Weight_Decay), init='normal',  activation='relu'))
model.add(permanent_dropout_layer(dropout,seed))
model.add(Dense(1, W_regularizer=l2(Weight_Decay), init='normal'))


model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=50, batch_size=batch_size, verbose=0)
All_Dropout_Scores = np.zeros(shape=(X_test.shape[0],1))
y_predicted = np.zeros(shape=(All_Dropout_Scores.shape[0]))
for d in range(dropout_iterations):
    dropout_score = predict_stochastic(model, X_test,batch_size=batch_size, verbose=0)
    All_Dropout_Scores = np.append(All_Dropout_Scores,dropout_score,axis=1)
for j in range(All_Dropout_Scores.shape[0]):
    P_temp = All_Dropout_Scores[j,:]
    P = np.delete(P_temp,0)
    P_mean = np.mean(P)
    y_predicted[j] = P_mean
mse = np.mean((y_test - y_predicted)**2)
print('r2 of test predictions: ' + str(r2_score(y_test, y_predicted)))
print('Mean Squared Error: ' + str(mse))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


r2 of test predictions: 0.49131683092128275
Mean Squared Error: 0.07257620067200239
